In [14]:
import pandas as pd
import plotly.express as px
import panel as pn
import os
from datetime import datetime

# Charger les données initiales
file_path = './dags/data/paris_wifi_data.csv'
data = pd.read_csv(file_path, sep=';')
data['cp'] = data['cp'].astype(str)
# Répertoire pour sauvegarder les plots
output_dir = './plots'
os.makedirs(output_dir, exist_ok=True)  # Crée le répertoire s'il n'existe pas

pn.extension('plotly')

# Fonction pour sauvegarder les graphiques avec gestion du nom et format PNG
def save_plot(fig, dashboard):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")  # Horodatage
    filename = f"{dashboard}_{timestamp}.png"  # Nom du fichier avec horodatage et format PNG
    fig.write_image(os.path.join(output_dir, filename))

# Graphique : Nombre de points WiFi par adresse
def plot_wifi_by_address():
    adresse_count = data['adresse'].value_counts().reset_index()
    adresse_count.columns = ['Adresse', 'Nombre de points WiFi']
    fig = px.bar(adresse_count, x='Adresse', y='Nombre de points WiFi', title="Nombre de points WiFi par adresse")
    fig.update_layout(xaxis_title="Adresse", yaxis_title="Nombre de points WiFi")
    save_plot(fig, "wifi_by_address")
    return fig

# Graphique : Nombre de points WiFi par code postal (cp)
def plot_wifi_by_cp():
    cp_count = data['cp'].value_counts().reset_index()
    cp_count.columns = ['cp', 'nombre_de_borne_wifi']
    fig = px.bar(cp_count, x='cp', y='nombre_de_borne_wifi', title="Répartition des points WiFi par code postal")
    fig.update_layout(xaxis_title="Code Postal", yaxis_title="Nombre de points WiFi")
    save_plot(fig, "wifi_by_cp")
    return fig

# Carte interactive : Emplacements des points WiFi
def plot_wifi_map():
    fig = px.scatter_mapbox(data, lat="latitude", lon="longitude", hover_name="nom_site",
                            title="Carte des points WiFi à Paris", mapbox_style="carto-positron", zoom=12)
    save_plot(fig, "wifi_map")
    return fig

# Graphique : Nombre de bornes WiFi par état pour chaque code postal (cp)
def plot_wifi_state_by_cp():
    state_count_cp = data.groupby(['cp', 'etat']).size().reset_index(name='Nombre de bornes')
    fig = px.bar(state_count_cp, x='cp', y='Nombre de bornes', color='etat',
                 title="Nombre de bornes par état pour chaque code postal")
    fig.update_layout(xaxis_title="Code Postal", yaxis_title="Nombre de bornes")
    save_plot(fig, "wifi_state_by_cp")
    return fig

# Fonction pour recharger les données
def reload_data():
    global data
    data = pd.read_csv(file_path)
    # Met à jour les graphiques avec les nouvelles données
    dashboard[1][0].object = plot_wifi_by_address()
    dashboard[1][1].object = plot_wifi_by_cp()
    dashboard[2].object = plot_wifi_map()
    dashboard[3][0].object = plot_wifi_state_by_site()
    dashboard[3][1].object = plot_wifi_state_by_cp()

# Configuration de l'actualisation périodique toutes les 5 minutes (300 secondes)
pn.state.add_periodic_callback(reload_data, period=300 * 1000)

# Création du tableau de bord avec Panel
dashboard = pn.Column(
    "# Tableau de bord des points WiFi à Paris",
    pn.Row(pn.panel(plot_wifi_by_address), pn.panel(plot_wifi_by_cp)),
    pn.panel(plot_wifi_map),
    pn.Row(pn.panel(plot_wifi_state_by_site), pn.panel(plot_wifi_state_by_cp))
)

# Affichage du tableau de bord
dashboard.show()


Launching server at http://localhost:57586
